# 汇率预测机器人

In [14]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

class CurrencyLSTMModel:
    def __init__(self, currency_name, data_path, look_back=1):
        self.currency_name = currency_name
        self.data_path = data_path
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_and_prepare_data(self):
     dataset = read_csv(self.data_path, header=0, index_col=0)
    # 直接用全部数据（只有汇率一列）
     values = dataset.values.astype('float32')
     scaled = self.scaler.fit_transform(values)
     reframed = series_to_supervised(scaled, self.look_back, 1)
     values = reframed.values
     n_train = int(len(values) * 0.7)
     train = values[:n_train, :]
     test = values[n_train:, :]
     train_X, train_y = train[:, :-1], train[:, -1]
     test_X, test_y = test[:, :-1], test[:, -1]
     train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],1))
     test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],1))
     return train_X, train_y, test_X, test_y

    def build_model(self, input_shape):
        model = Sequential()
        model.add(LSTM(6, input_shape=input_shape, return_sequences=True))
        model.add(LSTM(150, return_sequences=True))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        self.model = model

    def train(self, epochs=100, batch_size=60):
        train_X, train_y, test_X, test_y = self.load_and_prepare_data()
        self.build_model((train_X.shape[1], train_X.shape[2]))
        history = self.model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_X, test_y), verbose=2, shuffle=False)
        return history

    def save(self, model_path):
        self.model.save(model_path)

    def load(self, model_path):
        self.model = load_model(model_path)

    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat



## 新币汇率预测

In [26]:
import numpy as np
import pandas as pd
import joblib

sgp_model = CurrencyLSTMModel(currency_name='SGP', data_path='Exchange Rate SGP.csv', look_back=100)
sgp_model.train()
sgp_model.save('SGP_model.keras')
sgp_model.load('SGP_model.keras')

joblib.dump(sgp_model.scaler, 'scaler.save')
sgp_model.scaler = joblib.load('scaler.save')

# 读取原始数据
dataset = pd.read_csv('Exchange Rate SGP.csv', header=0, index_col=0)
values = dataset.values.astype('float32')


# 归一化
scaled = sgp_model.scaler.transform(values)

# 取最近100天
last_100 = scaled[-100:]  # shape: (100, 1)

# 递归预测未来20天
future_steps = 20
future_preds = []

input_seq = last_100.reshape((1, 100, 1))

for _ in range(future_steps):
    yhat = sgp_model.predict(input_seq)
    future_preds.append(yhat[0, 0])
    # 更新输入序列：去掉最早一天，加上最新预测
    new_input = np.append(input_seq[0, 1:, 0], yhat[0, 0])
    input_seq = new_input.reshape((1, 100, 1))

# 反归一化
future_preds = np.array(future_preds).reshape(-1, 1)
future_real = sgp_model.scaler.inverse_transform(future_preds)
print("未来20天预测汇率：")
print(future_real.flatten())


Epoch 1/100


c:\Users\wwwwang\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 2s - 951ms/step - loss: 0.2578 - val_loss: 0.1387
Epoch 2/100
2/2 - 0s - 100ms/step - loss: 0.1373 - val_loss: 0.1538
Epoch 3/100
2/2 - 0s - 101ms/step - loss: 0.0877 - val_loss: 0.2320
Epoch 4/100
2/2 - 0s - 100ms/step - loss: 0.1020 - val_loss: 0.1322
Epoch 5/100
2/2 - 0s - 105ms/step - loss: 0.0921 - val_loss: 0.1100
Epoch 6/100
2/2 - 0s - 102ms/step - loss: 0.0871 - val_loss: 0.1141
Epoch 7/100
2/2 - 0s - 103ms/step - loss: 0.0732 - val_loss: 0.1387
Epoch 8/100
2/2 - 0s - 103ms/step - loss: 0.0735 - val_loss: 0.1621
Epoch 9/100
2/2 - 0s - 98ms/step - loss: 0.0802 - val_loss: 0.1455
Epoch 10/100
2/2 - 0s - 100ms/step - loss: 0.0746 - val_loss: 0.1201
Epoch 11/100
2/2 - 0s - 101ms/step - loss: 0.0710 - val_loss: 0.1093
Epoch 12/100
2/2 - 0s - 100ms/step - loss: 0.0676 - val_loss: 0.1124
Epoch 13/100
2/2 - 0s - 104ms/step - loss: 0.0630 - val_loss: 0.1246
Epoch 14/100
2/2 - 0s - 104ms/step - loss: 0.0630 - val_loss: 0.1313
Epoch 15/100
2/2 - 0s - 100ms/step - loss: 0.0634 - val_